In [3]:
import tensorflow as tf
import keras
from keras.layers import Dense,Dropout, Activation
from keras.models import Sequential
from keras.optimizers import *

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [16]:
FILEROOT = "audio/"
labels = {'beach': 0,
          'bus': 1,
          'cafe/restaurant': 2,
          'car': 3,
          'city_center': 4,
          'forest_path': 5,
          'grocery_store': 6,
          'home': 7,
          'library': 8,
          'metro_station': 9,
          'office': 10,
          'park': 11,
          'residential_area': 12,
          'train': 13,
          'tram' : 14
          }
dev_files = pd.read_table(os.path.join(FILEROOT, "dev.txt"), names=['file', 'label'], sep='\s+')

for i, afile in dev_files.iterrows():
    y, sr = librosa.load(str(afile.file), sr=None)
    stft = np.abs(librosa.stft(y))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y, sr=sr).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).T,axis=0)
    
    line = np.concatenate([mfccs, chroma, mel, contrast, tonnetz]) 
    
    if i == 0:
        X_dev = [line]
        y_dev = [labels[afile.label]]
    else:
        X_dev += [line]
        y_dev += [labels[afile.label]]
    if ((i+1) % 10 == 0 or i+1 == test_files.shape[0]):
        print("Dev : {0}/{1}".format(i+1, test_files.shape[0]))
np.save("X_dev", X_dev)
np.save("y_dev", y_dev)

Dev : 10/298
Dev : 20/298
Dev : 30/298
Dev : 40/298
Dev : 50/298
Dev : 60/298
Dev : 70/298
Dev : 80/298
Dev : 90/298
Dev : 100/298
Dev : 110/298
Dev : 120/298
Dev : 130/298
Dev : 140/298
Dev : 150/298
Dev : 160/298
Dev : 170/298
Dev : 180/298
Dev : 190/298
Dev : 200/298
Dev : 210/298
Dev : 220/298
Dev : 230/298
Dev : 240/298
Dev : 250/298
Dev : 260/298
Dev : 270/298
Dev : 280/298
Dev : 290/298


In [5]:
FILEROOT = "audio/"

# ----------------------------------------------------------------------------

train_files = pd.read_table(os.path.join(FILEROOT, "train.txt"), names=['file', 'label'], sep='\s+')
test_files = pd.read_table(os.path.join(FILEROOT, "test_files.txt"), names=['file', 'label'], sep='\s+')

labels = {'beach': 0,
          'bus': 1,
          'cafe/restaurant': 2,
          'car': 3,
          'city_center': 4,
          'forest_path': 5,
          'grocery_store': 6,
          'home': 7,
          'library': 8,
          'metro_station': 9,
          'office': 10,
          'park': 11,
          'residential_area': 12,
          'train': 13,
          'tram' : 14
          }

for i, afile in train_files.iterrows():

    y, sr = librosa.load(str(afile.file), sr=None)
    
    stft = np.abs(librosa.stft(y))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y, sr=sr).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).T,axis=0)
    
    line = np.concatenate([mfccs, chroma, mel, contrast, tonnetz])    
    
    if i == 0:
        X_train = [line]
        y_train = [labels[afile.label]]
    else:
        X_train += [line]
        y_train += [labels[afile.label]]
    if ((i+1) % 10 == 0 or i+1 == train_files.shape[0]):
        print("Train : {0}/{1}".format(i+1, train_files.shape[0]))

"""for i, afile in test_files.iterrows():

    y, sr = librosa.load(str(afile.file), sr=None)
    mfcc = librosa.feature.mfcc(y=y, n_fft=512, hop_length=512, n_mfcc=40)
    # plt.figure(figsize=(10, 4))
    # librosa.display.specshow(librosa.power_to_db(np.abs(mfcc), ref=np.max),
    #                         y_axis='mel', x_axis='time')
    mfcc = np.mean(mfcc, axis=1)
    if i == 0:
        X_test = [mfcc]
    else:
        X_test += [mfcc]
    if ((i+1) % 100 == 0 or i+1 == test_files.shape[0]):
        print("Test : {0}/{1}".format(i+1, test_files.shape[0]))"""


X_tot = np.concatenate([X_train, X_test])
X_tot = preprocessing.scale(X_tot)
X_train = X_tot[:len(X_train)]
X_test = X_tot[len(X_train):]

Train : 10/582
Train : 20/582
Train : 30/582
Train : 40/582
Train : 50/582
Train : 60/582


c:\users\alexisr\appdata\local\programs\python\python35\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Train : 70/582
Train : 80/582
Train : 90/582
Train : 100/582
Train : 110/582
Train : 120/582
Train : 130/582
Train : 140/582
Train : 150/582
Train : 160/582
Train : 170/582
Train : 180/582
Train : 190/582
Train : 200/582
Train : 210/582
Train : 220/582
Train : 230/582
Train : 240/582
Train : 250/582
Train : 260/582
Train : 270/582
Train : 280/582
Train : 290/582
Train : 300/582
Train : 310/582
Train : 320/582
Train : 330/582
Train : 340/582
Train : 350/582
Train : 360/582
Train : 370/582
Train : 380/582
Train : 390/582
Train : 400/582
Train : 410/582
Train : 420/582
Train : 430/582
Train : 440/582
Train : 450/582
Train : 460/582
Train : 470/582
Train : 480/582
Train : 490/582
Train : 500/582
Train : 510/582
Train : 520/582
Train : 530/582
Train : 540/582
Train : 550/582
Train : 560/582
Train : 570/582
Train : 580/582
Train : 582/582


NameError: name 'X_test' is not defined

In [4]:
X_train = np.load("X_train.npy")

In [5]:
clf = ExtraTreesClassifier()
print(cross_val_score(clf,X_train,y_train,cv=5))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
np.savetxt("y_pred.txt", y_pred, fmt="%d")

NameError: name 'y_train' is not defined

In [17]:
y_trainonehot = pd.get_dummies(y_train)

In [25]:
model = Sequential()
model.add(Dense(100,input_dim = 193))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(15))
model.add(Activation('softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

TypeError: __init__() got an unexpected keyword argument 'learning_rate'

In [24]:
model.fit(X_train,y_trainonehot, batch_size = 8, epochs = 100, verbose = 1, validation_split = 0.2)
print(model.evaluate(X_train,y_trainonehot))

Train on 465 samples, validate on 117 samples
Epoch 1/100
465/465 [==============================] - 0s - loss: 0.1282 - acc: 0.9548 - val_loss: 4.4068 - val_acc: 0.4615
Epoch 2/100
465/465 [==============================] - 0s - loss: 0.1042 - acc: 0.9656 - val_loss: 4.7586 - val_acc: 0.3846
Epoch 3/100
465/465 [==============================] - 0s - loss: 0.1451 - acc: 0.9591 - val_loss: 3.8496 - val_acc: 0.3846
Epoch 4/100
465/465 [==============================] - 0s - loss: 0.1270 - acc: 0.9613 - val_loss: 4.1590 - val_acc: 0.3162

In [66]:
pred = model.predict(X_test)
print(pred.shape)
pred = [np.argmax(pred[i]) for i in range(len(pred))]
print(pred)
prec_values = np.loadtxt('y_pred.txt')
print(prec_values)
np.savetxt('y_pred.txt', pred, fmt="%d")


(298, 15)
[9, 8, 3, 7, 1, 5, 8, 14, 3, 7, 9, 0, 6, 14, 12, 5, 4, 8, 0, 9, 1, 6, 6, 0, 11, 12, 4, 7, 0, 3, 14, 1, 9, 11, 1, 5, 3, 3, 0, 4, 6, 0, 0, 6, 6, 11, 12, 6, 3, 8, 0, 9, 12, 12, 0, 9, 3, 6, 5, 6, 7, 4, 9, 0, 12, 13, 9, 8, 3, 4, 0, 6, 11, 8, 14, 2, 14, 10, 1, 9, 12, 0, 4, 2, 2, 3, 14, 13, 1, 7, 12, 14, 9, 6, 13, 14, 5, 9, 5, 13, 1, 13, 9, 6, 9, 0, 7, 6, 5, 14, 4, 12, 1, 2, 3, 0, 0, 9, 9, 9, 13, 3, 4, 11, 3, 1, 1, 12, 4, 5, 13, 8, 1, 9, 2, 12, 12, 0, 12, 14, 12, 5, 12, 9, 3, 3, 9, 5, 14, 3, 8, 3, 3, 0, 9, 13, 12, 0, 14, 6, 6, 4, 9, 9, 13, 1, 9, 5, 8, 7, 14, 13, 0, 0, 0, 14, 1, 12, 12, 12, 5, 6, 4, 2, 5, 6, 4, 14, 5, 10, 14, 6, 4, 8, 10, 3, 9, 12, 8, 0, 14, 9, 14, 3, 6, 0, 4, 14, 0, 4, 6, 6, 9, 9, 12, 2, 9, 14, 0, 14, 11, 6, 3, 4, 9, 12, 13, 14, 11, 4, 9, 0, 4, 0, 12, 9, 1, 5, 4, 12, 1, 8, 3, 11, 7, 5, 4, 6, 0, 1, 2, 2, 14, 9, 8, 5, 14, 5, 1, 3, 9, 6, 12, 12, 3, 12, 5, 0, 14, 4, 9, 3, 6, 14, 4, 11, 2, 14, 8, 5, 12, 5, 3, 14, 2, 6, 9, 14, 4, 0, 9, 8, 0, 9, 8, 9, 12, 12]
[  6.   8.   